&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../Start_Here.ipynb)

[Previous Notebook](Approach_to_the_Problem.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[1](Problem_statement.ipynb)
[2](Approach_to_the_Problem.ipynb)
[3]
[4](Visualising.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](Visualising.ipynb)


# ANI-1: an extensible neural network potential with DFT accuracy at force field computational cost - Part 3 

**Contents of the This Notebook:**

- [Getting Started](#Getting-Started)
- [AEV Computer and Loading data](#AEV-Computer-and-Loading-data)
- [Defining Networks for Atoms and Loss Function](#Defining-Networks-for-Atoms-and-Loss-Function)
- [Setting Hyper Parameters](#Setting-Hyper-Parameters)
- [Training](#Training)
- [Visualisations]()


**By the End of this Notebook you will:**

- Define and Train the Model.

# Getting Started

Let us start this off by importing the necessary libraries.

In [ ]:
#Let's Import Required Modules
import torch
import torchani
import os
import math
import torch.utils.tensorboard
import tqdm

# device to run the training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# AEV Computer and Loading data

Next we need to construct the AEV Computer using certain values calculated from the Dataset.

In [ ]:
# Setup Parameters Computer from the ANI1 Dataset
Rcr = 5.2000e+00
Rca = 3.5000e+00
EtaR = torch.tensor([1.6000000e+01], device=device)
ShfR = torch.tensor([9.0000000e-01, 1.1687500e+00, 1.4375000e+00, 1.7062500e+00, 1.9750000e+00, 2.2437500e+00, 2.5125000e+00, 2.7812500e+00, 3.0500000e+00, 3.3187500e+00, 3.5875000e+00, 3.8562500e+00, 4.1250000e+00, 4.3937500e+00, 4.6625000e+00, 4.9312500e+00], device=device)
Zeta = torch.tensor([3.2000000e+01], device=device)
ShfZ = torch.tensor([1.9634954e-01, 5.8904862e-01, 9.8174770e-01, 1.3744468e+00, 1.7671459e+00, 2.1598449e+00, 2.5525440e+00, 2.9452431e+00], device=device)
EtaA = torch.tensor([8.0000000e+00], device=device)
ShfA = torch.tensor([9.0000000e-01, 1.5500000e+00, 2.2000000e+00, 2.8500000e+00], device=device)
num_species = 4

# Setup AEV Computer
aev_computer = torchani.AEVComputer(Rcr, Rca, EtaR, ShfR, EtaA, Zeta, ShfA, ShfZ, num_species)

# Setting Parameter to `None` will allow EnergyShifter to Calculate Self Energy of the Atoms
energy_shifter = torchani.utils.EnergyShifter(None)

#Convert Chemical Symbols to Integers
species_to_tensor = torchani.utils.ChemicalSymbolsToInts('HCNO')

Also note that we need to subtracting energies by the self energies of all
atoms for each molecule. This makes the range of energies in a reasonable
range. The second argument defines how to convert species as a list of string
to tensor, that is, for all supported chemical symbols, which is correspond to
``0``, which correspond to ``1``, etc.

In [ ]:
try:
    path = os.path.dirname(os.path.realpath(__file__))
except NameError:
    path = os.getcwd()
dspath = os.path.join(path, 'ANI1_release/ani_gdb_s04.h5')

batch_size = 2560

training, validation = torchani.data.load_ani_dataset(
    dspath, species_to_tensor, batch_size, rm_outlier=True, device=device,
    transform=[energy_shifter.subtract_from_dataset], split=[0.8, None])

print('Self atomic energies: ', energy_shifter.self_energies)

When iterating the dataset, we will get pairs of input and output
 ``(species_coordinates, properties)``, where ``species_coordinates`` is the
 input and ``properties`` is the output.

 ``species_coordinates`` is a list of species-coordinate pairs, with shape
 ``(N, Na)`` and ``(N, Na, 3)``. The reason for getting this type is, when
 loading the dataset and generating minibatches, the whole dataset are
 shuffled and each minibatch contains structures of molecules with a wide
 range of number of atoms. Molecules of different number of atoms are batched
 into single by padding. The way padding works is: adding ghost atoms, with
 species 'X', and do computations as if they were normal atoms. But when
 computing AEVs, atoms with species `X` would be ignored. To avoid computation
 wasting on padding atoms, minibatches are further splitted into chunks. Each
 chunk contains structures of molecules of similar size, which minimize the
 total number of padding atoms required to add. The input list
 ``species_coordinates`` contains chunks of that minibatch we are getting. The
 batching and chunking happens automatically, so the user does not need to
 worry how to construct chunks, but the user need to compute the energies for
 each chunk and concat them into single tensor.

 The output, i.e. ``properties`` is a dictionary holding each property. This
 allows us to extend TorchANI in the future to training forces and properties.


Now let's define atomic neural networks.



# Defining Networks for Atoms and Loss Function 

We will implement the Mean Squared Error Loss from the PyTorch library

In [ ]:
mse = torch.nn.MSELoss(reduction='none')

This network is Simple Feed-forward Network with Inputs as 384 and outputs the Energy of the Atom.

In [ ]:
H_network = torch.nn.Sequential(
    torch.nn.Linear(384, 160),
    torch.nn.CELU(0.1),
    torch.nn.Linear(160, 128),
    torch.nn.CELU(0.1),
    torch.nn.Linear(128, 96),
    torch.nn.CELU(0.1),
    torch.nn.Linear(96, 1)
)

C_network = torch.nn.Sequential(
    torch.nn.Linear(384, 144),
    torch.nn.CELU(0.1),
    torch.nn.Linear(144, 112),
    torch.nn.CELU(0.1),
    torch.nn.Linear(112, 96),
    torch.nn.CELU(0.1),
    torch.nn.Linear(96, 1)
)

N_network = torch.nn.Sequential(
    torch.nn.Linear(384, 128),
    torch.nn.CELU(0.1),
    torch.nn.Linear(128, 112),
    torch.nn.CELU(0.1),
    torch.nn.Linear(112, 96),
    torch.nn.CELU(0.1),
    torch.nn.Linear(96, 1)
)

O_network = torch.nn.Sequential(
    torch.nn.Linear(384, 128),
    torch.nn.CELU(0.1),
    torch.nn.Linear(128, 112),
    torch.nn.CELU(0.1),
    torch.nn.Linear(112, 96),
    torch.nn.CELU(0.1),
    torch.nn.Linear(96, 1)
)

nn = torchani.ANIModel([H_network, C_network, N_network, O_network])
print(nn)

Initiliase Weights and biases :

In [ ]:
# Let us Now Initialise the Weights and biases
def init_params(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight, a=1.0)
        torch.nn.init.zeros_(m.bias)


nn.apply(init_params)

Let's now create a pipeline of AEV Computer --> Neural Networks.


In [ ]:
model = torchani.nn.Sequential(aev_computer, nn).to(device)

Now let's setup the optimizers. NeuroChem uses Adam with decoupled weight decay
to updates the weights and Stochastic Gradient Descent (SGD) to update the biases.
Moreover, we need to specify different weight decay rate for different layes.


# Setting Hyper Parameters

In [ ]:
AdamW = torchani.optim.AdamW([
    # H networks
    {'params': [H_network[0].weight]},
    {'params': [H_network[2].weight], 'weight_decay': 0.00001},
    {'params': [H_network[4].weight], 'weight_decay': 0.000001},
    {'params': [H_network[6].weight]},
    # C networks
    {'params': [C_network[0].weight]},
    {'params': [C_network[2].weight], 'weight_decay': 0.00001},
    {'params': [C_network[4].weight], 'weight_decay': 0.000001},
    {'params': [C_network[6].weight]},
    # N networks
    {'params': [N_network[0].weight]},
    {'params': [N_network[2].weight], 'weight_decay': 0.00001},
    {'params': [N_network[4].weight], 'weight_decay': 0.000001},
    {'params': [N_network[6].weight]},
    # O networks
    {'params': [O_network[0].weight]},
    {'params': [O_network[2].weight], 'weight_decay': 0.00001},
    {'params': [O_network[4].weight], 'weight_decay': 0.000001},
    {'params': [O_network[6].weight]},
])

SGD = torch.optim.SGD([
    # H networks
    {'params': [H_network[0].bias]},
    {'params': [H_network[2].bias]},
    {'params': [H_network[4].bias]},
    {'params': [H_network[6].bias]},
    # C networks
    {'params': [C_network[0].bias]},
    {'params': [C_network[2].bias]},
    {'params': [C_network[4].bias]},
    {'params': [C_network[6].bias]},
    # N networks
    {'params': [N_network[0].bias]},
    {'params': [N_network[2].bias]},
    {'params': [N_network[4].bias]},
    {'params': [N_network[6].bias]},
    # O networks
    {'params': [O_network[0].bias]},
    {'params': [O_network[2].bias]},
    {'params': [O_network[4].bias]},
    {'params': [O_network[6].bias]},
], lr=1e-3)

Setting up a learning rate scheduler to do learning rate decay



In [ ]:
AdamW_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(AdamW, factor=0.5, patience=100, threshold=0)
SGD_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(SGD, factor=0.5, patience=100, threshold=0)

Train the model by minimizing the MSE loss, until validation RMSE no longer
improves during a certain number of steps, decay the learning rate and repeat
the same process, stop until the learning rate is smaller than a threshold.


During training, we need to validate on validation set and if validation error
is better than the best, then save the new best model to a checkpoint



In [ ]:
# helper function to convert energy unit from Hartree to kcal/mol
def hartree2kcal(x):
    return 627.509 * x


def validate():
    # run validation
    mse_sum = torch.nn.MSELoss(reduction='sum')
    total_mse = 0.0
    count = 0
    for batch_x, batch_y in validation:
        true_energies = batch_y['energies']
        predicted_energies = []
        for chunk_species, chunk_coordinates in batch_x:
            chunk_energies = model((chunk_species, chunk_coordinates)).energies
            predicted_energies.append(chunk_energies)
        predicted_energies = torch.cat(predicted_energies)
        total_mse += mse_sum(predicted_energies, true_energies).item()
        count += predicted_energies.shape[0]
    return hartree2kcal(math.sqrt(total_mse / count))

We will also use TensorBoard to visualize our training process



In [ ]:
tensorboard = torch.utils.tensorboard.SummaryWriter()

# Training

Finally, we come to the training loop.

In this tutorial, we are setting the maximum epoch to a small number,
only to make this demo terminate fast. For proper training, this should be
set to a much larger value



In [ ]:
# Training starts here
print("training starting from epoch", 0 + 1)
# Set Parameters - Max_Epochs & Early Stopping 
max_epochs = 10
early_stopping_learning_rate = 1.0E-5
# Store the Best Model as a Checkpoint
best_model_checkpoint = 'best.pt'

# Loop through the Training Epoch
for _ in range(AdamW_scheduler.last_epoch + 1, max_epochs):
    rmse = validate()
    print('RMSE:', rmse, 'at epoch', AdamW_scheduler.last_epoch + 1)

    learning_rate = AdamW.param_groups[0]['lr']
    
    # Stop training if Learning Rate is Less than Early Stopping Rate 
    if learning_rate < early_stopping_learning_rate:
        break

    # Store the Best Model
    if AdamW_scheduler.is_better(rmse, AdamW_scheduler.best):
        torch.save(nn.state_dict(), best_model_checkpoint)
    
    # Learning Rate Scheduler 
    AdamW_scheduler.step(rmse)
    SGD_scheduler.step(rmse)
    
    #Add Parameters to Tensorbaord 
    tensorboard.add_scalar('validation_rmse', rmse, AdamW_scheduler.last_epoch)
    tensorboard.add_scalar('best_validation_rmse', AdamW_scheduler.best, AdamW_scheduler.last_epoch)
    tensorboard.add_scalar('learning_rate', learning_rate, AdamW_scheduler.last_epoch)
    
    # Loop through the batch of Data
    for i, (batch_x, batch_y) in tqdm.tqdm(
        enumerate(training),
        total=len(training),
        desc="epoch {}".format(AdamW_scheduler.last_epoch)
    ):
        # Store Predicted Values
        true_energies = batch_y['energies']
        predicted_energies = []
        num_atoms = []

        for chunk_species, chunk_coordinates in batch_x:
            num_atoms.append((chunk_species >= 0).to(true_energies.dtype).sum(dim=1))
            chunk_energies = model((chunk_species, chunk_coordinates)).energies
            predicted_energies.append(chunk_energies)

        num_atoms = torch.cat(num_atoms)
        predicted_energies = torch.cat(predicted_energies)
        loss = (mse(predicted_energies, true_energies) / num_atoms.sqrt()).mean()
        
        # Zero out the Gradients and Run Backpropgation. 
        AdamW.zero_grad()
        SGD.zero_grad()
        loss.backward()
        AdamW.step()
        SGD.step()

        # write current batch loss to TensorBoard
        tensorboard.add_scalar('batch_loss', loss, AdamW_scheduler.last_epoch * len(training) + i)

# Training Visulisations 

Open `Tensorboard` from Jupyter Home to analyse training data.

### In the Next Notebook , Let us see how this model inference is used in Molecular Dynamics.

[Previous Notebook](Approach_to_the_Problem.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[1](Problem_statement.ipynb)
[2](Approach_to_the_Problem.ipynb)
[3]
[4](Visualising.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[Next Notebook](Visualising.ipynb)

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../Start_Here.ipynb)